In [1]:
import pandas as pd
import numpy as np

import random
from itertools import product

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from catboost import CatBoostRegressor
from neuralprophet import NeuralProphet


import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error

from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib

import os
import shutil
import tempfile

import gc

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


### download data

In [2]:
# Download Data Dictionary
data_dict_path = '../../data/data_dictionary.xlsx'
data_dictionary = pd.read_excel(data_dict_path)

# Download Scoring File -- what the input data will be for the end
scoring_path = '../../data/scoring.xlsx'
scoring = pd.read_excel(scoring_path)

# Download Submission Format -- what the format of the output should be from my model
submission_format_path = '../../data/submission_format.csv'
submission_format = pd.read_csv(submission_format_path)

# Download Training Data -- what the training data is for this
training_path = '../../data/training.xlsx'
training = pd.read_excel(training_path).dropna()

# new data from Fred: https://fred.stlouisfed.org/series/CUSR0000SETA02
cpi_used_cars_path = '../../data/CPI_UsedCars_US.xlsx'
cpi_used_cars = pd.read_excel(cpi_used_cars_path, sheet_name='Monthly')
cpi_used_cars.columns = ['Date', 'CPI']
cpi_used_cars['Year'] = cpi_used_cars['Date'].dt.year
average_cpi_by_year = cpi_used_cars.groupby('Year')['CPI'].mean().reset_index().rename(columns={'Year': 'Model Year'})
# since average cpi by year is missing 2025 and 2026, forecast using a neural prophet model
def forecast_cpi_polynomial(average_cpi_by_year, forecast_periods=5, degree=2):
    # Ensure data is sorted by 'Model Year'
    average_cpi_by_year = average_cpi_by_year.sort_values(by='Model Year')

    # Extract features and target variable
    X = average_cpi_by_year[['Model Year']]
    y = average_cpi_by_year['CPI']

    # Generate polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)

    # Fit the polynomial regression model
    model = LinearRegression()
    model.fit(X_poly, y)

    # Generate future 'Model Year' values
    last_year = X['Model Year'].max()
    future_years = np.arange(last_year + 1, last_year + 1 + forecast_periods).reshape(-1, 1)

    # Transform future years to polynomial features
    future_years_poly = poly.transform(future_years)

    # Predict future CPI values
    future_cpi = model.predict(future_years_poly)

    # Create DataFrame for future predictions
    future_df = pd.DataFrame({
        'Model Year': future_years.flatten(),
        'CPI': future_cpi
    })

    # Combine original and forecasted data
    combined_df = pd.concat([average_cpi_by_year, future_df], ignore_index=True)

    return combined_df

average_cpi_by_year = forecast_cpi_polynomial(average_cpi_by_year, 2, 4)


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(



In [3]:
def clean_input_data(data):
    # add the CPI of Used Cars by City Average
    clean_data = data.merge(average_cpi_by_year, on='Model Year', how='left')
    clean_data['Model Year'] = clean_data['Model Year'].astype(str)
    
    # handle nulls
    clean_data = clean_data.replace({"nan": np.nan})
    clean_data['Model Year'] = clean_data['Model Year'].fillna("Missing")

    # fill missing CPI's with average CPI
    clean_data['CPI'] = clean_data['CPI'].fillna(clean_data['CPI'].mean())
    
    return clean_data

### model functions

In [4]:
def fit_linear_regression(X, y):
    model = LinearRegression()
    model.fit(X, y)

    return model

class ConstantModel:
    # A fallback model that always predicts a constant value
    def __init__(self, value):
        self.value = value  # Store the constant target value

    def predict(self, X):
        #  mimic contant
        if not hasattr(X, '__len__'):  # Ensure X has a length (is iterable)
            return self.value  # Just return the value if X isn't iterable
        return [self.value] * len(X)  # Normal case


def fit_catboost(X, y):
    try:

        categorical_cols = list(X.columns)
        if 'CPI' in categorical_cols:
            categorical_cols.remove('CPI')

        model = CatBoostRegressor(
            iterations=100,
            learning_rate=0.05,
            depth=6,
            cat_features=categorical_cols,
            random_seed=42,
            allow_writing_files=False,  # Prevents CatBoost from creating the catboost_info directory
            verbose=0
        )

        model.fit(X, y)

        # Clear CatBoost temporary files
        catboost_tmp_dir = tempfile.gettempdir()  # Get system temp dir
        for root, dirs, files in os.walk(catboost_tmp_dir):
            for dir_name in dirs:
                if "catboost" in dir_name:
                    shutil.rmtree(os.path.join(root, dir_name), ignore_errors=True)

        return model
    except:
        singular_value = y.iloc[0] if hasattr(y, 'iloc') else y[0]
        return ConstantModel(singular_value)
    


In [5]:
def clear_cache_and_temp_files(root_dir='.'):
    """
    Recursively deletes __pycache__ directories and .pyc files starting from the root_dir.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # Remove __pycache__ directories
        if '__pycache__' in dirnames:
            pycache_path = os.path.join(dirpath, '__pycache__')
            shutil.rmtree(pycache_path, ignore_errors=True)
            print(f"Deleted directory: {pycache_path}")
        
        # Remove .pyc files
        for filename in filenames:
            if filename.endswith('.pyc'):
                file_path = os.path.join(dirpath, filename)
                os.remove(file_path)
                print(f"Deleted file: {file_path}")

### download train and test data

In [6]:
clean_training = clean_input_data(training)
clean_scoring = clean_input_data(scoring)

car_data = pd.concat([clean_training, clean_scoring]).reset_index(drop=True)

categorical_cols = car_data.select_dtypes(include=['object', 'category']).columns.tolist()
encoded_car_data =  pd.get_dummies(car_data, columns=categorical_cols)

train_indices = range(0, len(clean_training))
test_indices = range(len(clean_training), len(car_data))

train = car_data.loc[train_indices].copy()
test = car_data.loc[test_indices].copy()

train_encoded = encoded_car_data.loc[train_indices].copy()
test_encoded = encoded_car_data.loc[test_indices].copy()

### model testing functions

In [7]:
def get_models_by_basis(car_data, encoded_car_data, train_indices, basis_columns, prediction_columns, target_col, model_func, encode):

    train_data = car_data.loc[train_indices]
    encoded_train = encoded_car_data.loc[train_indices]

    # Define the columns used to create different models
    basis_values = [train_data[col].unique().tolist() for col in basis_columns] # these are the potential values for each basis

    # Generate all possible (Model Year, Fuel Type) combinations as tuples
    basis_combinations = [tuple(values) for values in product(*basis_values)] # these are the combos of basis's so like 2020 Electric for example

    # Dictionary to store trained models, using (Model Year, Fuel Type) as the key
    trained_models = {}
    for basis_key in basis_combinations:

        basis_dict = {col: value for col, value in zip(basis_columns, basis_key)}
        # Filter training data for the specific Model Year & Fuel Type
        basis_indices = train_data[
            train_data[basis_dict.keys()].eq(pd.Series(basis_dict)).all(axis=1)
        ].index 

        # check if encoded
        model_data = encoded_train.copy() if encode else train_data.copy()

        # initialize X and y
        X = model_data.loc[basis_indices, prediction_columns] if len(basis_indices) != 0 else pd.DataFrame()
        y = model_data.loc[basis_indices][target_col] if len(basis_indices) != 0 else pd.DataFrame()


        if len(X) > 10: # if there are at least 10 values
            model = model_func(X, y)
            safe = True
        else:
            model = model_func(model_data[prediction_columns], model_data[target_col])
            safe = False

        # Store trained model using the (Model Year, Fuel Type) tuple as the key
        trained_models[basis_key] = (model, safe)

    return trained_models

In [8]:
def calculate_test_rmse(car_data, encoded_car_data, test_indices, trained_models, basis_columns, prediction_columns, target_col, encode):
    all_prediction_rows = []
    test = car_data.loc[test_indices]

    for test_idx in test_indices:

        row = car_data.loc[test_idx]

        # get the basis values in the row (like 2020 for Model Year if Model Year is a basis)
        model_basis_dict = {col: row[col] for col in basis_columns}

        # get the model based on the basis
        model, safe = trained_models[tuple(model_basis_dict.values())]

        # get the encoded row
        if encode:
            formatted_row = encoded_car_data.loc[test_idx]
            formatted_row = formatted_row[prediction_columns]
        else:
            formatted_row = row[prediction_columns]

        # get the prediction vs. actual
        prediction = model.predict(pd.DataFrame(formatted_row).T)[0]
        actual = test.loc[test_idx, target_col]

        prediction_row = pd.DataFrame({
            'index': [test_idx],
            'prediction': [round(prediction, 0)],
            'actual': [actual],
            'safe': [safe]
        })
        all_prediction_rows.append(prediction_row)

    prediction_df = pd.concat(all_prediction_rows)
    rmse = np.sqrt(mean_squared_error(prediction_df['actual'], prediction_df['prediction']))

    return rmse, prediction_df

### model testing

###### Get the combinations of basis columns and prediction columns

In [9]:
column_options = list(set(car_data.columns) - {'Date', 'Vehicle Population'})

basis_prediction_combos = []
for _ in range(250):
    # Select 1-3 random basis columns
    basis_columns = random.sample(column_options, random.randint(1, 2))
    if 'CPI' in basis_columns:
        basis_columns.remove('CPI')
    if len(basis_columns) == 0:
        continue
    
    # Select 3-7 prediction columns that are **not in basis_columns**
    remaining_columns = list(set(column_options) - set(basis_columns))
    prediction_columns = random.sample(remaining_columns, random.randint(3, 8-len(basis_columns)))
    
    # Store as a tuple
    basis_prediction_combos.append((basis_columns, prediction_columns))


###### Run the tests

In [10]:
# Define function to process each set of columns
model_func = fit_catboost
encode = False

# Define file path for saving results
results_file = '../../results/catboost_optim.csv'

# Load existing results if the file exists
if os.path.exists(results_file):
    model_results = pd.read_csv(results_file)
else:
    model_results = pd.DataFrame(columns=['basis_columns', 'prediction_columns', 'rmse'])

# Convert existing basis-prediction combinations to a set for quick lookup
existing_combos = set(zip(model_results['basis_columns'].astype(str), model_results['prediction_columns'].astype(str)))

target_col = 'Vehicle Population'

def process_combo(basis_columns, prediction_columns, save=True):
    try:
        
        # Convert combo to string format for checking existence
        combo_key = (str(basis_columns), str(prediction_columns))
        if combo_key in existing_combos:
            print(f"Skipping already processed: {basis_columns}, {prediction_columns}")
            return None

        # Ensure prediction columns do not include basis columns
        if encode:
            clean_prediction_columns = list(set(col for col in train_encoded.columns if col.split("_")[0] not in basis_columns) - {target_col, 'Date'})
        else:
            clean_prediction_columns = prediction_columns
        
        # Train models and calculate RMSE
        trained_models = get_models_by_basis(car_data, encoded_car_data, train_indices, basis_columns, clean_prediction_columns, target_col, model_func, encode)
        rmse = calculate_test_rmse(car_data, encoded_car_data, test_indices, trained_models, basis_columns, clean_prediction_columns, target_col, encode)[0]

        # Create DataFrame for this iteration
        result_df = pd.DataFrame({
            'basis_columns': [basis_columns],
            'prediction_columns': [prediction_columns],
            'rmse': [rmse]
        })

        if save:
            # Append results to CSV immediately
            result_df.to_csv(results_file, index=False, mode='a', header=not os.path.exists(results_file))

        try:
            shutil.rmtree('catboost_info')
        except:
            pass

        return result_df
    except Exception as e:
        print(f"Exception: {e}")
        return None

# Initialize tqdm progress bar
with tqdm_joblib(tqdm(desc="Processing Models", total=len(basis_prediction_combos))) as progress_bar:
    results = Parallel(n_jobs=-1)(
        delayed(process_combo)(basis, pred) for basis, pred in basis_prediction_combos
    )

ParserError: Error tokenizing data. C error: Expected 3 fields in line 64, saw 5


### run model on new data

In [ ]:
encoded_scoring =  encoded_car_data.loc[test_indices]

# initialize inputs
model_func = fit_catboost
encode = False
basis_columns = ['Vehicle Category', 'Fuel Type']
prediction_columns = ['GVWR Class', 'CPI', 'Number of Vehicles Registered at the Same Address', 'Model Year', 'Fuel Technology']
target_col = 'Vehicle Population'

# clean prediction columns if necessary
if encode:
    clean_prediction_columns = list(set(col for col in encoded_scoring.columns if col.split("_")[0] not in basis_columns) - {target_col, 'Date'})
else:
    clean_prediction_columns = prediction_columns

trained_models = get_models_by_basis(car_data, encoded_car_data, train_indices, basis_columns, clean_prediction_columns, target_col, model_func, encode)
rmse, pred_df = calculate_test_rmse(car_data, encoded_car_data, test_indices, trained_models, basis_columns, clean_prediction_columns, target_col, encode)

submission = pred_df[['prediction']].rename(columns={'prediction': 'Vehicle Population'}).reset_index(drop=True)
rmse

Processing Models:   0%|          | 0/1 [00:53<?, ?it/s]


9160.065809717278